In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime  
from nltk.tokenize import word_tokenize  
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind
import numpy as np
import re

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder

from sklearn.metrics import accuracy_score,roc_auc_score, f1_score, recall_score
from sklearn.metrics import  roc_curve, confusion_matrix, precision_score



from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer

# function to make n-grams
from nltk.util import ngrams 
from nltk import everygrams

from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(vocabulary = myvocabulary, stop_words = 'english')
# tfs = tfidf.fit_transform(corpus.values())

from gensim.corpora import Dictionary
import itertools
from collections import defaultdict
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore') 
%matplotlib inline

In [3]:
df = pd.read_csv('../data/processed/data_for_modelling.csv')

# TF-IDF Vectorization

In [4]:
# instantiate the tf-idf object
tfidf_vectors = TfidfVectorizer(max_df=0.90, min_df=2, max_features=9000, 
                                stop_words='english',
                                ngram_range=(1, 3))



In [5]:
# TF-IDF feature matrix
tfidf = tfidf_vectors.fit_transform(df['tweet_without_stopwords_and_2'])

In [6]:
df_vector = pd.DataFrame(tfidf.todense(),columns = tfidf_vectors.get_feature_names())

# Splitting into Training/Validation/Tes

In [7]:
target = df['pos_label']


In [8]:
x_train, x_test, y_train , y_test = train_test_split(df_vector, target, 
                                                     test_size =.2, random_state=101 )

In [9]:
print(f'The shape of my training set is {x_train.shape}')
print(f'The shape of my training target is {y_train.shape}')
print(f'The shape of my test set is {x_test.shape}')
print(f'The shape of my test target is {y_test.shape}')

The shape of my training set is (33593, 9000)
The shape of my training target is (33593,)
The shape of my test set is (8399, 9000)
The shape of my test target is (8399,)


In [10]:
X_train, x_val, Y_train , y_val = train_test_split(x_train,y_train, 
                                                     test_size =.2, random_state=101 )

In [11]:
print(f'The shape of my final training set is {X_train.shape}')
print(f'The shape of my final training target is {Y_train.shape}')
print(f'The shape of my validation set is {x_val.shape}')
print(f'The shape of my validation target is {y_val.shape}')

The shape of my final training set is (26874, 9000)
The shape of my final training target is (26874,)
The shape of my validation set is (6719, 9000)
The shape of my validation target is (6719,)


# Logistic Regression Baseline

In [12]:
logmodel = LogisticRegression()

In [13]:
logmodel.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
# testing on validation
y_hat_log = logmodel.predict(x_test)





In [15]:
f1_score(y_test, y_hat_log) # calculating f1 score

0.9818390804597701

In [16]:


confusion_matrix(y_test, y_hat_log)

array([[3970,  122],
       [  36, 4271]])

In [17]:
tn, fp, fn, tp = confusion_matrix(y_test, y_hat_log).ravel()
tn, fp, fn, tp

(3970, 122, 36, 4271)

In [18]:
y_hat_proba = logmodel.predict_proba(x_test)

In [20]:
y_hat_proba[:10]

array([[0.06976815, 0.93023185],
       [0.13261807, 0.86738193],
       [0.98345116, 0.01654884],
       [0.0489999 , 0.9510001 ],
       [0.93550274, 0.06449726],
       [0.13404968, 0.86595032],
       [0.89883245, 0.10116755],
       [0.87324718, 0.12675282],
       [0.03572845, 0.96427155],
       [0.03958806, 0.96041194]])

In [21]:
y_test

21489    1
14714    1
12192    0
31163    1
6053     0
        ..
28287    0
21944    1
26081    1
32660    0
2474     0
Name: pos_label, Length: 8399, dtype: int64

In [23]:
Y_hat_proba = y_hat_proba[:,1]

In [25]:
Y_hat_proba[:10]

array([0.93023185, 0.86738193, 0.01654884, 0.9510001 , 0.06449726,
       0.86595032, 0.10116755, 0.12675282, 0.96427155, 0.96041194])

In [26]:
class_neg = Y_hat_proba > .7

In [27]:
class_neg

array([ True,  True, False, ...,  True, False, False])